In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import fsspec
import time

import numpy as np
import pandas as pd
import xarray as xr

from carbonplan_trace.v1.glas_preprocess import *
from carbonplan_trace.v1.glas_height_metrics import *
from carbonplan_trace.v1.glas_allometric_eq import * 


In [3]:
url01 = f'gs://carbonplan-scratch/trace_scratch/wa_glah01.zarr'
mapper01 = fsspec.get_mapper(url01)
data01 = xr.open_zarr(mapper01)

url14 = f'gs://carbonplan-scratch/trace_scratch/wa_glah14.zarr'
mapper14 = fsspec.get_mapper(url14)
data14 = xr.open_zarr(mapper14)

combined = data14.merge(data01)

/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 93 times more chunks
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 93 times more chunks
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 55 times more chunks
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 93 times more chunks
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 93 times more chunks
  return self.array[key]
/srv/conda

In [4]:
combined

<xarray.Dataset>
Dimensions:                   (rec_bin: 544, record_index: 25307, shot_number: 40, tx_bin: 48)
Coordinates:
  * record_index              (record_index) int64 22232240 ... 1069303896
  * shot_number               (shot_number) int64 1 2 3 4 5 6 ... 36 37 38 39 40
  * rec_bin                   (rec_bin) int64 0 1 2 3 4 ... 539 540 541 542 543
  * tx_bin                    (tx_bin) int64 0 1 2 3 4 5 6 ... 42 43 44 45 46 47
Data variables:
    centroid_offset           (record_index, shot_number) float64 dask.array<chunksize=(6327, 10), meta=np.ndarray>
    elevation                 (record_index, shot_number) float64 dask.array<chunksize=(6327, 10), meta=np.ndarray>
    elevation_correction      (record_index, shot_number) float64 dask.array<chunksize=(6327, 10), meta=np.ndarray>
    lat                       (record_index, shot_number) float64 dask.array<chunksize=(6327, 10), meta=np.ndarray>
    lon                       (record_index, shot_number) float64 dask.array<chunksize=(6327, 10), meta=np.ndarray>
    ref_range                 (record_index, shot_number) float64 dask.array<chunksize=(6327, 10), meta=np.ndarray>
    sig_begin_offset          (record_index, shot_number) float64 dask.array<chunksize=(6327, 10), meta=np.ndarray>
    sig_end_offset            (record_index, shot_number) float64 dask.array<chunksize=(6327, 10), meta=np.ndarray>
    time                      (record_index, shot_number) float64 dask.array<chunksize=(6327, 10), meta=np.ndarray>
    noise_mean                (record_index, shot_number) float64 dask.array<chunksize=(44, 10), meta=np.ndarray>
    noise_sd                  (record_index, shot_number) float64 dask.array<chunksize=(44, 10), meta=np.ndarray>
    rec_wf                    (rec_bin, record_index, shot_number) float64 dask.array<chunksize=(34, 44, 5), meta=np.ndarray>
    rec_wf_location_ind       (record_index, shot_number) float64 dask.array<chunksize=(44, 10), meta=np.ndarray>
    rec_wf_response_end_time  (record_index, shot_number) float64 dask.array<chunksize=(44, 10), meta=np.ndarray>
    rec_wf_sample_loc         (rec_bin, record_index, shot_number) float64 dask.array<chunksize=(34, 44, 5), meta=np.ndarray>
    tx_wf                     (tx_bin, record_index, shot_number) float64 dask.array<chunksize=(6, 44, 10), meta=np.ndarray>
    tx_wf_peak_time           (record_index, shot_number) float64 dask.array<chunksize=(44, 10), meta=np.ndarray>
    tx_wf_sample_loc          (tx_bin, record_index, shot_number) float64 dask.array<chunksize=(6, 44, 10), meta=np.ndarray>

In [6]:
def main(ds):
    # generic processing 
    print('Starting preprocess')
    t1 = time.time()
    ds = preprocess(ds).compute()
    
    t2 = time.time()
    print(f'Finished preprocess, took {(t2 - t1)/60.} mins')
    # TODO: save the processed waveforms 
    
    # specific height metrics (still in units of "distance from satellite")
    ds['meanH_distance'] = get_mean_distance(
        bins=ds.rec_wf_sample_distance,
        wf=ds.processed_wf
    ).compute()
    
    # percentile distance 
    percentiles = [10, 90]  # get 10th and 90th percentiles 
    percentile_distances = get_percentile_distance(
        bins=ds.rec_wf_sample_distance,
        wf=ds.processed_wf,
        percentiles=percentiles,
    ).compute()
    for p in percentiles:
        ds[f'{p}th_distance'] = percentile_distances[p]
    
    # get ground peak distance 
    ds['ground_peak_distance'] = get_ground_peak_distance(
        bins=ds.rec_wf_sample_distance, 
        wf=ds.processed_wf
    ).compute()
    
    # get heights from distance
    list_of_distance_vars = [
        'meanH_distance',
        '10th_distance',
        '90th_distance',
    ]
    ds = get_heights_from_distance(
        ds=ds,
        list_of_distance_vars=list_of_distance_vars,
        referece_distance_var='ground_peak_distance'
    ).compute()
    t3 = time.time()
    print(f'Finished getting height metrics, took {(t3 - t2)/60.} mins')
    
    ds = apply_allometric_equation(ds).compute()
    t4 = time.time()
    print(f'Finished getting height metrics, took {(t4 - t3)/60.} mins')
    
    return ds

# could start a cluster 
# ds.to_zarr('cloud')

In [ ]:
combined = main(combined).compute()

Starting preprocess
Using default sigma
Using default sigma
Using default sigma


In [ ]:
record = combined.sel(record_index=22232245, shot_number=33)
plot_shot(record)

In [ ]:
combined.biomass.values

In [ ]:
url = f'gs://carbonplan-scratch/trace_scratch/wa_biomass.zarr'
mapper = fsspec.get_mapper(url)
combined.to_zarr(mapper)

In [76]:
from pyproj import transform


In [94]:
def transform_lat_lon(lat, lon):
    return np.array(transform(4326, 32610, lat, lon))


In [99]:
xr.apply_ufunc(
    transform_lat_lon,
    sub.lat,
    sub.lon,
    vectorize=True,
    dask='parallelized',
    dask_gufunc_kwargs={'allow_rechunk': 1},
    output_core_dims=[['lat_lon']],
    output_sizes=
    output_dtypes=np.float64
).compute()

ValueError: dimension 'lat_lon' in 'output_core_dims' needs corresponding (dim, size) in 'output_sizes'

In [58]:
df = pd.read_csv('index.csv')
df.head()

x0, y0, x1, y1 = [-124.763068, 45.543541, -116.915989, 49.002494]
time_start = '2003-02-20T00:00:00Z'
time_end = '2009-10-11T23:59:59Z'

df_wa = df[(time_start < df['SENSING_TIME']) &
           (df['SENSING_TIME'] < time_end) &
           (df['NORTH_LAT'] < y1) &
           (df['SOUTH_LAT'] > y0) &
           (df['WEST_LON'] > x0) &
           (df['EAST_LON'] < x1)]

In [62]:
landsat_times = pd.to_datetime(df_wa.SENSING_TIME.sort_values()

6579444   2003-02-22 18:44:18.623309800+00:00
5418066   2003-02-24 18:31:56.330728300+00:00
7951453      2003-03-02 18:29:07.800006+00:00
4226349   2003-03-03 18:38:11.174776400+00:00
7881665      2003-03-04 18:16:49.970006+00:00
                          ...                
6584803      2009-09-26 18:45:33.521019+00:00
4043452   2009-09-27 18:39:36.026604300+00:00
7362237      2009-09-28 18:33:13.353013+00:00
6821768   2009-10-04 18:45:45.487984200+00:00
1231777      2009-10-05 18:39:28.772094+00:00
Name: SENSING_TIME, Length: 862, dtype: datetime64[ns, UTC]

In [68]:
ds = xr.DataArray(dims=['landsat_time'], coords=[landsat_times])

In [70]:
def get_nearest_time(glastime, list_of_time):
    # find nearest 
    
xr.apply_ufunc(
    get_nearest_time,
    sub.time,
    ds
)

<xarray.Dataset>
Dimensions:                   (rec_bin: 544, record_index: 3, shot_number: 40, tx_bin: 48)
Coordinates:
  * record_index              (record_index) int64 22232240 22232245 22232250
  * shot_number               (shot_number) int64 1 2 3 4 5 6 ... 36 37 38 39 40
  * rec_bin                   (rec_bin) int64 0 1 2 3 4 ... 539 540 541 542 543
  * tx_bin                    (tx_bin) int64 0 1 2 3 4 5 6 ... 42 43 44 45 46 47
Data variables:
    centroid_offset           (record_index, shot_number) float64 dask.array<chunksize=(3, 10), meta=np.ndarray>
    elevation                 (record_index, shot_number) float64 dask.array<chunksize=(3, 10), meta=np.ndarray>
    elevation_correction      (record_index, shot_number) float64 dask.array<chunksize=(3, 10), meta=np.ndarray>
    lat                       (record_index, shot_number) float64 dask.array<chunksize=(3, 10), meta=np.ndarray>
    lon                       (record_index, shot_number) float64 dask.array<chunksize=(3, 10), meta=np.ndarray>
    ref_range                 (record_index, shot_number) float64 dask.array<chunksize=(3, 10), meta=np.ndarray>
    sig_begin_offset          (record_index, shot_number) float64 dask.array<chunksize=(3, 10), meta=np.ndarray>
    sig_end_offset            (record_index, shot_number) float64 dask.array<chunksize=(3, 10), meta=np.ndarray>
    time                      (record_index, shot_number) float64 dask.array<chunksize=(3, 10), meta=np.ndarray>
    noise_mean                (record_index, shot_number) float64 dask.array<chunksize=(3, 10), meta=np.ndarray>
    noise_sd                  (record_index, shot_number) float64 dask.array<chunksize=(3, 10), meta=np.ndarray>
    rec_wf                    (rec_bin, record_index, shot_number) float64 dask.array<chunksize=(34, 3, 5), meta=np.ndarray>
    rec_wf_location_ind       (record_index, shot_number) float64 dask.array<chunksize=(3, 10), meta=np.ndarray>
    rec_wf_response_end_time  (record_index, shot_number) float64 dask.array<chunksize=(3, 10), meta=np.ndarray>
    rec_wf_sample_loc         (rec_bin, record_index, shot_number) float64 dask.array<chunksize=(34, 3, 5), meta=np.ndarray>
    tx_wf                     (tx_bin, record_index, shot_number) float64 dask.array<chunksize=(6, 3, 10), meta=np.ndarray>
    tx_wf_peak_time           (record_index, shot_number) float64 dask.array<chunksize=(3, 10), meta=np.ndarray>
    tx_wf_sample_loc          (tx_bin, record_index, shot_number) float64 dask.array<chunksize=(6, 3, 10), meta=np.ndarray>
    rec_wf_sample_distance    (rec_bin, record_index, shot_number) float64 dask.array<chunksize=(34, 3, 5), meta=np.ndarray>
    ref_range_bias            (record_index, shot_number) float64 dask.array<chunksize=(3, 5), meta=np.ndarray>
    sig_begin_distance        (record_index, shot_number) float64 dask.array<chunksize=(3, 5), meta=np.ndarray>
    sig_end_distance          (record_index, shot_number) float64 dask.array<chunksize=(3, 5), meta=np.ndarray>
    centroid_distance         (record_index, shot_number) float64 dask.array<chunksize=(3, 5), meta=np.ndarray>
    processed_wf              (record_index, shot_number, rec_bin) float64 dask.array<chunksize=(3, 5, 34), meta=np.ndarray>

In [57]:
# for each glas 

ds.sel(landsat_time=glas_time, method='nearest')

# avail_times = list_of_time 

# sub.sel(time=1, method=

# i have list of available timestamps 
# for each record tell me which time is closest 


array([[              nan,               nan,               nan,
                      nan,               nan,               nan,
                      nan,               nan,               nan,
                      nan,               nan,               nan,
                      nan,               nan,               nan,
                      nan,               nan,               nan,
                      nan,               nan,               nan,
                      nan,               nan,               nan,
                      nan,               nan,               nan,
                      nan,               nan,               nan,
                      nan, 99097648.84339401, 99097648.868394  ,
        99097648.89339401, 99097648.918394  , 99097648.943394  ,
        99097648.96839401, 99097648.993394  , 99097649.01839401,
        99097649.043394  ],
       [99097649.068394  , 99097649.09339401, 99097649.118394  ,
        99097649.14339401, 99097649.168394  , 99097649.193394 

In [53]:
combined.tx_wf_sample_loc

,Array,Chunk
Bytes,388.72 MB,86.40 kB
Shape,"(48, 25307, 40)","(6, 180, 10)"
Count,14209 Tasks,13952 Chunks
Type,float64,numpy.ndarray


In [30]:
# TODO
# 1) find all the unique rec_wf_sample_loc forms, and make sure that if the compression ratio changed, it changed before signal beginning distance 
# 2) 

<xarray.DataArray (record_index: 25307, shot_number: 40)>
dask.array<sub, shape=(25307, 40), dtype=float64, chunksize=(180, 5), chunktype=numpy.ndarray>
Coordinates:
  * record_index  (record_index) int64 22232240 22232245 ... 1069303896
  * shot_number   (shot_number) int64 1 2 3 4 5 6 7 8 ... 34 35 36 37 38 39 40

In [ ]:
def interpolate_wf(bins, wf, target, area_to_include):
    """
    not vectorized 
    if area_to_include = 'above', the function returns the area between the target location to the "upper bound" (larger value) bin
    if area_to_include = 'below', the function returns the area between the target location to the "lower bound" (smaller value) bin
    """        
    upper_ind = np.where(bins > target)[0].max()
    lower_ind = np.where(bins < target)[0].min()
    # since bins goes from large to small values, the "upper bound" index would be smaller than the "lower bound" index
    assert lower_ind - upper_ind == 1
    
    x_upper = bins[upper_ind]
    x_lower = bins[lower_ind]
    y_upper = wf[upper_ind]
    y_lower = wf[lower_ind]
    
    x_mid = (x_upper + x_lower) / 2.
    x_span = x_upper - x_lower 
    
    if area_to_include == 'above':
        if target < x_mid:
            energy = (x_mid - target) / x_span * y_lower  # energy to add to the lower bin
            bin_to_modify = lower_ind
        else:
            energy = (x_mid - target) / x_span * y_upper  # energy to subtract out of the upper bin
            bin_to_modify = upper_ind 
    elif area_to_include == 'below':
        if target < x_mid:
            energy = (target - x_mid) / x_span * y_lower  # energy to subtract out of the lower bin
            bin_to_modify = lower_ind
        else:
            energy = (target - x_mid) / x_span * y_upper  # energy to add to the upper bin
            bin_to_modify = upper_ind
    else:
        raise NotImplementedError('Please specify whether we want to include area above or below the target to the bounds')
                
    return bin_to_modify, energy

def interpolate_and_select_valid_area(bins, wf, beg, end):
    """
    not vectorized
    """
    # initialize output 
    output = np.zeros(len(wf))
    
    # within signal beginning and end locations, set otuput to be equal to input wf
    valid = np.where((bins > beg) & (bins < end))[0]
    output[valid] = wf[valid]
    
    # for the begining and end bin, interpolate 
    # bins goes from large values (furthest away from satellite) to small (closest to satellite)
    bin_to_modify, energy = interpolate_wf(bins, wf, beg, 'above')
    bins[bin_to_modify] += energy
    bin_to_modify, energy = interpolate_wf(bins, wf, end, 'below')
    bins[bin_to_modify] += energy
    
    # min at 0
    output = np.maximum(output, 0)
    
    return output

